In [0]:
%sql
SELECT 'device_messages_raw' AS table_name, COUNT(*) AS rows FROM workspace.bronze.device_messages_raw;

In [0]:
%sql
SELECT device_id, COUNT(*) AS messages
FROM workspace.bronze.device_messages_raw
GROUP BY device_id
ORDER BY messages DESC
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW device_messages_clean AS
SELECT
  device_id,
  TRY_CAST(REGEXP_REPLACE(CAST(distance AS STRING), '[^0-9.]', '') AS DOUBLE) AS distance_cm,
  CAST(timestamp AS BIGINT) AS ts_ms,
  CAST(date AS BIGINT) AS date_ms,
  message_origin,
  sensor_type,
  message
FROM workspace.bronze.device_messages_raw;

In [0]:
%sql
SELECT * FROM device_messages_clean LIMIT 20;

In [0]:
%sql
SELECT device_id,
       AVG(distance_cm) AS avg_cm,
       MIN(distance_cm) AS min_cm,
       MAX(distance_cm) AS max_cm
FROM device_messages_clean
GROUP BY device_id
ORDER BY avg_cm DESC
LIMIT 10;

In [0]:
%sql
WITH exploded AS (
  SELECT
    customer,
    device_id,
    start_time,
    posexplode(step_points) AS (step_index, step_ms)
  FROM workspace.bronze.rapid_step_tests_raw
)
SELECT customer, device_id,
       COUNT(*) AS steps,
       AVG(step_ms) AS avg_step_ms,
       STDDEV(step_ms) AS sd_step_ms
FROM exploded
GROUP BY customer, device_id
ORDER BY steps DESC
LIMIT 10;

This query shows how many sensor readings occurred during each test and the average/min/max distance for that window. We will reuse these aggregates as features for ML later.